In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
import pinecone

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

In [3]:
PINECONE_API_KEY = "2636083b-4c32-4640-95a4-536e2f08b060"
PINECONE_API_ENV = "gcp-starter"

In [4]:
# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    
    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 ,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [8]:
# text_chunks

In [9]:
# Downloading the Embedding model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [12]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))
print()
print(query_result)

Length 384

[-0.03447724133729935, 0.031023146584630013, 0.006734973285347223, 0.026108982041478157, -0.03936201333999634, -0.16030243039131165, 0.06692401319742203, -0.0064414553344249725, -0.04745049774646759, 0.01475889328867197, 0.07087531685829163, 0.05552763491868973, 0.01919335499405861, -0.026251377537846565, -0.010109560564160347, -0.026940448209643364, 0.022307464852929115, -0.022226646542549133, -0.1496926248073578, -0.017493000254034996, 0.007676302921026945, 0.05435224995017052, 0.0032544273417443037, 0.03172589838504791, -0.08462139219045639, -0.029405996203422546, 0.05159558355808258, 0.048124056309461594, -0.0033147973008453846, -0.05827922374010086, 0.041969262063503265, 0.022210683673620224, 0.1281888484954834, -0.02233891189098358, -0.011656300164759159, 0.06292836368083954, -0.03287629410624504, -0.09122603386640549, -0.031175360083580017, 0.05269954726099968, 0.04703474044799805, -0.0842030718922615, -0.030056197196245193, -0.020744869485497475, 0.00951785501092672

In [13]:
pinecone.init(      
	api_key=PINECONE_API_KEY,      
	environment=PINECONE_API_ENV      
)      
index_name = 'chatbot-medical'

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings,index_name=index_name)

In [14]:
# We already have an index we cam load it like this
docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query,k=3)

print("Result",docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which co

In [15]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs = {"prompt":PROMPT}


In [17]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type='llama',
                    config={'max_new_tokens':512,
                            'temperature':0.8})


In [18]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [19]:
#while True:
#    user_input = input(f"Input Prompt :")
#    result = qa({"query": user_input})
#    print("Response: ",result["result"])